In [2]:
import pyspark

In [3]:
pyspark.__version__

'3.3.2'

In [4]:
pyspark.__file__

'/mnt/c/Users/D.Bosyak/source/git/data-engineering-zoomcamp/spark/spark-3.3.2-bin-hadoop3/python/pyspark/__init__.py'

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder  \
        .master("local[*]")  \
        .appName('test')  \
        .getOrCreate()

23/03/05 10:03:04 WARN Utils: Your hostname, N-OFFICE-894 resolves to a loopback address: 127.0.1.1; using 172.24.26.10 instead (on interface eth0)
23/03/05 10:03:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/05 10:03:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [46]:
df_zone = spark.read.option("header", "true").csv('taxi+_zone_lookup.csv')
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [7]:
df_zone.write.parquet('zones')

AnalysisException: path file:/mnt/c/Users/D.Bosyak/source/git/data-engineering-zoomcamp/spark/jupyter_env/zones already exists.

In [8]:
ls zones/

_SUCCESS*  part-00000-fcc7a59a-0d51-4628-89fe-6f869710253c-c000.snappy.parquet*


In [ ]:
!wc -l fhvhv_tripdata_2021-06.csv.gz

In [ ]:
!gzip -d fhvhv_tripdata_2021-06.csv.gz

In [9]:
df = spark.read  \
    .option("header", "true")  \
    .csv('fhvhv_tripdata_2021-06.csv')

df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
|              B02510|2021-06-01 00:18:1

In [10]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [ ]:
!head -n 1001 fhvhv_tripdata_2021-06.csv > head.csv

In [49]:
import pandas as pd

In [50]:
df_pandas = pd.read_csv('head.csv')

In [51]:
df_pandas.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [52]:
from pyspark.sql import types

In [53]:
df_schema = types.StructType([
                        types.StructField("dispatching_base_num", types.StringType(), True),  
                        types.StructField("pickup_datetime", types.TimestampType(), True),  
                        types.StructField("dropoff_datetime", types.TimestampType(), True),  
                        types.StructField("PULocationID", types.IntegerType(), True),  
                        types.StructField("DOLocationID", types.IntegerType(), True),  
                        types.StructField("SR_Flag", types.StringType(), True), 
                        types.StructField("Affiliated_base_number", types.StringType(), True)
])

In [54]:
df = spark.read  \
    .option("header", "true")  \
    .schema(df_schema)  \
    .csv('fhvhv_tripdata_2021-06.csv')

In [55]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
|              B02510|2021-06-01 00:18:1

In [19]:
df = df.repartition(12)

In [20]:
df.write.parquet('fhvhv/2021/06')

23/03/04 20:24:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [18]:
df = spark.read.parquet('fhvhv/2021/06')

In [19]:
df

DataFrame[dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string, Affiliated_base_number: string]

In [20]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [21]:
from pyspark.sql import functions as F

In [22]:
df_newColumn =  \
df.withColumn('pickup_date', F.to_date(df.pickup_datetime))  \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime))  \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID')

In [23]:
df_newColumn.show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-06-03|  2021-06-03|          72|          76|
| 2021-06-02|  2021-06-02|         198|         198|
| 2021-06-01|  2021-06-01|          68|         231|
| 2021-06-02|  2021-06-02|          41|         119|
| 2021-06-01|  2021-06-01|         230|          90|
| 2021-06-04|  2021-06-04|         188|          37|
| 2021-06-04|  2021-06-04|          68|          50|
| 2021-06-04|  2021-06-04|         249|          88|
| 2021-06-04|  2021-06-04|         234|         170|
| 2021-06-03|  2021-06-03|         167|         265|
| 2021-06-03|  2021-06-03|          39|          39|
| 2021-06-02|  2021-06-02|          42|         212|
| 2021-06-03|  2021-06-03|          79|          79|
| 2021-06-01|  2021-06-01|         219|         265|
| 2021-06-03|  2021-06-03|          76|          36|
| 2021-06-03|  2021-06-03|         141|       

In [24]:
df_newColumn.filter(df_newColumn.pickup_date == '2021-06-15').groupBy('pickup_date').count().show()

+-----------+------+
|pickup_date| count|
+-----------+------+
| 2021-06-15|452470|
+-----------+------+



In [25]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02887|2021-06-03 18:00:43|2021-06-03 18:11:49|          72|          76|      N|                B02887|
|              B02884|2021-06-02 14:37:26|2021-06-02 14:44:02|         198|         198|      N|                B02884|
|              B02882|2021-06-01 18:23:50|2021-06-01 18:45:24|          68|         231|      N|                B02882|
|              B02510|2021-06-02 23:16:01|2021-06-02 23:32:58|          41|         119|      N|                  null|
|              B02510|2021-06-01 17:36:28|2021-06-01 17:50:42|         230|          90|      N|                  null|
|              B02878|2021-06-04 21:48:4

In [37]:
df  \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long'))  \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime))  \
    .groupBy('pickup_date').max('duration')  \
    .orderBy('max(duration)', ascending=False)  \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2021-06-25|       240764|
| 2021-06-22|        91979|
| 2021-06-27|        71931|
| 2021-06-26|        65510|
| 2021-06-23|        59281|
| 2021-06-24|        50075|
| 2021-06-04|        42012|
| 2021-06-20|        39544|
| 2021-06-01|        36963|
| 2021-06-28|        35879|
| 2021-06-18|        34648|
| 2021-06-08|        34129|
| 2021-06-11|        34098|
| 2021-06-15|        33848|
| 2021-06-03|        33717|
| 2021-06-19|        32785|
| 2021-06-30|        32602|
| 2021-06-09|        32509|
| 2021-06-17|        31587|
| 2021-06-29|        30858|
+-----------+-------------+
only showing top 20 rows



In [38]:
df.registerTempTable('fhvhv_2021_06')

/mnt/c/Users/D.Bosyak/source/git/data-engineering-zoomcamp/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [44]:
spark.sql("""
SELECT 
    to_date(pickup_datetime) AS pickup_date
  , MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 60 / 60) AS duration
FROM fhvhv_2021_06
GROUP BY to_date(pickup_datetime)
ORDER BY duration DESC;
""").show(10)

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2021-06-25| 66.87888888888888|
| 2021-06-22|25.549722222222222|
| 2021-06-27|19.980833333333333|
| 2021-06-26| 18.19722222222222|
| 2021-06-23|16.466944444444444|
| 2021-06-24|13.909722222222223|
| 2021-06-04|             11.67|
| 2021-06-20|10.984444444444446|
| 2021-06-01|           10.2675|
| 2021-06-28|  9.96638888888889|
+-----------+------------------+
only showing top 10 rows



In [57]:
df_preJoin = df.withColumn('LocationID', df.PULocationID)
df_preJoin.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|LocationID|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|       174|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|        32|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|       240|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|       127|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|        

In [59]:
df_join = df_preJoin.join(df_zone, on=['LocationID'], how='outer')

In [60]:
df_join.show()

+----------+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+---------+------------+------------+
|LocationID|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|  Borough|        Zone|service_zone|
+----------+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+---------+------------+------------+
|        12|              B02887|2021-06-01 00:18:57|2021-06-01 00:28:29|          12|           4|      N|                B02887|Manhattan|Battery Park| Yellow Zone|
|        12|              B02876|2021-06-01 09:21:27|2021-06-01 09:58:24|          12|         197|      N|                B02876|Manhattan|Battery Park| Yellow Zone|
|        12|              B02872|2021-06-01 11:53:08|2021-06-01 12:17:48|          12|          74|      N|                B02872|Manhattan|Battery Park| Yellow Zone

In [61]:
df_join.registerTempTable('fhvhv_2021_06_frequent_zone')

/mnt/c/Users/D.Bosyak/source/git/data-engineering-zoomcamp/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [62]:
spark.sql("""
SELECT 
    LocationID AS LocationID
  , Zone AS Zone
  , COUNT(1) AS pickup_count
FROM fhvhv_2021_06_frequent_zone
GROUP BY LocationID, Zone
ORDER BY pickup_count DESC;
""").show(10)

23/03/05 11:57:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:00 WARN RowBasedKeyValueBatch: Calling spill() on

23/03/05 11:57:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 11:57:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/03/05 11:57:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+----------+--------------------+------------+
|LocationID|                Zone|pickup_count|
+----------+--------------------+------------+
|        61| Crown Heights North|      231279|
|        79|        East Village|      221244|
|       132|         JFK Airport|      188867|
|        37|      Bushwick South|      187929|
|        76|       East New York|      186780|
|       231|TriBeCa/Civic Center|      164344|
|       138|   LaGuardia Airport|      161596|
|       234|            Union Sq|      158937|
|       249|        West Village|      154698|
|         7|             Astoria|      152493|
+----------+--------------------+------------+
only showing top 10 rows

